<a href="https://colab.research.google.com/github/MaseedIrfanAli/DataFlow/blob/main/ApachePCollect1_Avro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WriteToAvro(file_path_prfix,Schema,codee(block),file_name_suffix,numshards,shard_name_template,mime_type,use_fastro(fastest avro file handle)
)

WriteToParquet(file_path_prfix,Schema,row_group_buffer_size,record_batch_size,codec(blocksizecompression),use_deprecated_int_96_timestamp(everyelementsavewithtimestamp
),file_name_suffix,numshards,shard_name_template,mime_type(multipurposemailextension)
               )

Columner storage of parquet files;

data is row formate  row1 row2 row3
data is columner col1 col2 col3

in memory data
- - - - - - - - - - - -
- - - - - - - - - - - -

so data will fill in  left to right here

Here Parquet

File
______________
row group 0
______________
name column Chunk

age column Chunk

Grade Column Chunk

______________
row group 1
______________
name column Chunk

age column Chunk

Grade Column Chunk





In [4]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 16.3 MB/s eta 0:00:00
 

In [16]:
import apache_beam as beam
p1 = beam.Pipeline()
customers = (
    p1
    |beam.io.ReadFromText('/content/Irfan/Customer.txt')
    |beam.Map(lambda record:record.split(','))
    |beam.Filter(lambda record: record[2]=='New York' and int(record[3])>20)
    |beam.io.WriteToText('/content/Irfan/result')
)
p1.run()


In [17]:
!cat /content/Irfan/result-00000-of-00001

['1', ' John Doe', 'New York', ' 30']
['5', ' David Brown', 'New York', ' 28']


"Peter Piper picked a peck of pickled peppers.
A peck of pickled peppers Peter Piper picked.
If Peter Piper picked a peck of pickled peppers,
Where's the peck of pickled peppers Peter Piper picked?"

In [24]:
import apache_beam as beam
words=['peter','piper','pickled','picked','peck','pepper']
p1 = beam.Pipeline()
def FindWord(element):
  if element in words:
    return True
Freq = (
    p1
    |beam.io.ReadFromText('/content/Irfan/Peter_piper.txt')
    |beam.FlatMap(lambda record:record.split(' '))
    |beam.Filter(FindWord)
    |beam.Map(lambda record: (record,1))
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('/content/Irfan/result')
)
p1.run()


In [25]:
!cat /content/Irfan/result-00000-of-00001


('picked', 2)
('peck', 4)
('pickled', 4)


CoGroupByKey
pcol1 pcol2
jhon,2   jhon,988982

output:pcol1+pcol2=(jhon,{[2],[988982]})

In [29]:
import apache_beam as beam

p = beam.Pipeline()
movie_name = [
    (1, 'SpiderMan'),
    (2, 'Avenger'),
    (3, 'Titanic'),
    (4, 'Green Miles'),
]
movies_rating = [
    (1, 3.5),
    (2, 4),
    (1,4.5),
    (3,3.5),
    (2,4.5)
]

name = p | 'Create Name Pcolletion' >> beam.Create(movie_name)
rating = p | 'Create Rating Pcolletion' >> beam.Create(movies_rating)

joinedResult = ({'movie_name' : name, 'movie_rating':rating} | beam.CoGroupByKey()) | beam.Map(print)

p.run()


(1, {'movie_name': ['SpiderMan'], 'movie_rating': [3.5, 4.5]})
(2, {'movie_name': ['Avenger'], 'movie_rating': [4, 4.5]})
(3, {'movie_name': ['Titanic'], 'movie_rating': [3.5]})
(4, {'movie_name': ['Green Miles'], 'movie_rating': []})


Partition

In [30]:
import apache_beam as beam
p1=beam.Pipeline()

number={1,2,3,4,5,6,7,8,9,10}
def partition_fn(element,num_partitions):
  return 0 if element%2==0 else 1

num_pc=(
    p1
    |beam.Create(number)
    |beam.Partition(partition_fn,2)
    )
num_pc[0]|"Prining first partition" >> beam.Map(print)

p1.run()

2
4
6
8
10


In [31]:
import apache_beam as beam
p1=beam.Pipeline()

number={1,2,3,4,5,6,7,8,9,10}
def partition_fn(element,num_partitions):
  return 0 if element%2==0 else 1

num_pc=(
    p1
    |beam.Create(number)
    |beam.Partition(partition_fn,2)
    )
num_pc[1]|"Prining first partition" >> beam.Map(print)

p1.run()

1
3
5
7
9


Flatten transformation(Union)
wanted to combine all

In [33]:
# -*- coding: utf-8 -*-
"""Lecture12_Flatten.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ZSp_mz8UC7fmSqKSL8qGAp7-QlfKFXLy
"""



import apache_beam as beam

p = beam.Pipeline()

even = {2,4,6,8}
odd = {1,3,5,7,9}
name = ('John','Jim','Mary')

even_pc = p | 'Create Pcollection from Even number' >> beam.Create(even)
odd_pc = p | 'Create Pcollection from Odd Number' >> beam.Create(odd)
name_pc = p | 'Create Pcollection from Name' >> beam.Create(name)

result = ((even_pc,odd_pc,name_pc) | beam.Flatten()) | beam.Map(print)
p.run()

John
Jim
Mary
8
2
4
6
1
3
5
7
9


Composite Transformation:
why to existing metheds to one method

In [66]:


import apache_beam as beam
p1 = beam.Pipeline()

class MyTransform(beam.PTransform):
  def expand(self, input_col):
    a = (
        input_col
                   | 'Calculate Sum' >> beam.Map(CalculateSum)
                   | 'Apply Formatting' >> beam.Map(FormatText)
    )
    return a

def SplitRow(input_element):
  return input_element.split(',')

def FilterBasedonCountry(countryName,input_element):
  return input_element[1]==countryName

def CalculateSum(elem):
  return elem[0],(int(elem[2])+int(elem[3])+int(elem[4]))

def FormatText(elem):
  return elem[0]+' has recieved '+str(elem[1]) +' marks'

input_collection = (
    p1
    |beam.io.ReadFromText('/content/Irfan/Student_marks.txt')
    |beam.Map(SplitRow)
  )

US_pipeline = (
   input_collection
   | beam.Filter(lambda record: FilterBasedonCountry('US',record))
   | "Composite Transformation for US" >> MyTransform()
   | 'Writing results to US File' >> beam.io.WriteToText('/content/Irfan/US_Result')
)

India_pipeline = (
   input_collection
   | beam.Filter(lambda record: FilterBasedonCountry('IN',record))
   | "Composite Transformation for IN" >> MyTransform()
   |'Writing results to India File' >> beam.io.WriteToText('/content/Irfan/IN_Result')
)


p1.run()

! cat US_Result-00000-of-00001

! cat IN_Result-00000-of-00001

cat: US_Result-00000-of-00001: No such file or directory
cat: IN_Result-00000-of-00001: No such file or directory


In [69]:
!cat /content/Irfan/US_Result-00000-of-00001